In [1]:
import os
os.chdir("../")
%pwd

'd:\\DataScience\\Thyroid-disease-prediction'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    subsample: float
    gamma: int
    target_column: str

In [20]:
from Thyroid_Disease.constants import *
from Thyroid_Disease.utils.common import read_yaml, create_directories

In [21]:
# configuration manager
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path= config.test_data_path,
            model_name=config.model_name,
            subsample= params.subsample,
            gamma = params.gamma,
            target_column= schema.name,
        )

        create_directories([config.root_dir])

        return model_trainer_config

In [22]:
import pandas as pd
import os
from Thyroid_Disease import logger
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import joblib

In [23]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        model = XGBClassifier(subsample=self.config.subsample, gamma=self.config.gamma)
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))



In [24]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-04-04 00:21:33,539 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-04-04 00:21:33,548 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-04-04 00:21:33,564 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-04-04 00:21:33,568 : INFO : common : created directory at: artifacts]
[2024-04-04 00:21:33,572 : INFO : common : created directory at: artifacts/model_trainer]
